In [38]:
from utils import get_possible_hands
import pandas as pd
import numpy as np
from constants import HOLE_CARD_DICT
from constants import HOLE_CARDS

In [39]:
equity_data = pd.read_csv("data/equity_data.csv")

In [66]:
equity_data['hand1.hand'] = equity_data['hand1.hand'].apply(
        lambda x: x if x.endswith(('o', 's')) else x + 'o'
    )

equity_data['hand2.hand'] = equity_data['hand2.hand'].apply(
        lambda x: x if x.endswith(('o', 's')) else x + 'o'
    )